<a href="https://colab.research.google.com/github/polinomas1/face-demographics/blob/main/face_demographics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deepface tf-keras opencv-python pandas tqdm matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.1/197.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 4.4 MB/s eta 0:00:00


In [2]:
from google.colab import files
import os

# Sukuriame aplankus
os.makedirs('data', exist_ok=True)
os.makedirs('results', exist_ok=True)

print("Įkelkite savo vaizdo įrašą:")
uploaded = files.upload()

# Paimame įkelto failo pavadinimą
input_filename = next(iter(uploaded))

# Perkeliame į data aplanką
os.rename(input_filename, f"data/{input_filename}")
print(f"Failas išsaugotas kaip: data/{input_filename}")

Įkelkite savo vaizdo įrašą:


Saving stock-footage-aged-man-shopper-choose-bargain-point-finger-isolated-bright-color-background.mp4 to stock-footage-aged-man-shopper-choose-bargain-point-finger-isolated-bright-color-background.mp4
Failas išsaugotas kaip: data/stock-footage-aged-man-shopper-choose-bargain-point-finger-isolated-bright-color-background.mp4


In [3]:
import cv2
import pandas as pd
from deepface import DeepFace
from tqdm import tqdm
import os, time
import glob

INPUT_FILE = f"data/{input_filename}"

# Surandam sekantį run numerį pagal jau esančius CSV
existing = glob.glob("results/analysis_data_*.csv")

nums = []
for f in existing:
    name = os.path.splitext(os.path.basename(f))[0]  # analysis_data_3
    try:
        nums.append(int(name.split("_")[-1]))
    except:
        pass

run_id = (max(nums) + 1) if nums else 1

OUTPUT_VIDEO = f"results/output_video_{run_id}.mp4"
OUTPUT_CSV = f"results/analysis_data_{run_id}.csv"

def process_video():
    cap = cv2.VideoCapture(INPUT_FILE)

    if not cap.isOpened():
        print("Klaida: Nepavyko atidaryti video failo.")
        return

    # Video parametrai
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Išvesties nustatymas (MP4 formatas tinka Colab/Web)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (width, height))

    results = []
    print(f"Pradedama analizė. Iš viso kadrų: {total_frames}")

    # DeepFace kartais užtrunka inicijuojant, todėl pirmas kadras gali būti lėtas
    pbar = tqdm(total=total_frames)

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        try:
            # Analizė. enforce_detection=False svarbu, jei kadre nėra veido
            # detector_backend='opencv' yra greičiausias, 'retinaface' tiksliausias bet lėtas
            faces = DeepFace.analyze(img_path=frame,
                                     actions=['age', 'gender'],
                                     enforce_detection=False,
                                     detector_backend='retinaface',
                                     silent=True)

            for face in faces:
                region = face['region']
                x, y, w, h = region['x'], region['y'], region['w'], region['h']
                age = face['age']
                gender = face['dominant_gender']

                # Kaupiame duomenis
                results.append({
                    'frame': frame_idx,
                    'timestamp': frame_idx/fps,
                    'age': age,
                    'gender': gender
                })

                # Piešiame ant kadro
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                label = f"{gender}, {int(age)}"
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        except Exception as e:
            pass # Jei klaida - praleidžiam kadrą

        out.write(frame)
        pbar.update(1)
        frame_idx += 1

    cap.release()
    out.release()
    pbar.close()

    # Išsaugom CSV
    df = pd.DataFrame(results)
    df.to_csv(OUTPUT_CSV, index=False)
    print("\nBaigta!")

# Paleidžiam funkciją
process_video()

26-02-14 15:40:19 - Directory /root/.deepface has been created
26-02-14 15:40:19 - Directory /root/.deepface/weights has been created
Pradedama analizė. Iš viso kadrų: 461


  0%|          | 0/461 [00:00<?, ?it/s]Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5


26-02-14 15:40:23 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5



  0%|          | 0.00/119M [00:00<?, ?B/s]
 23%|██▎       | 27.3M/119M [00:00<00:00, 272MB/s]
 61%|██████▏   | 72.9M/119M [00:00<00:00, 380MB/s]
100%|██████████| 119M/119M [00:00<00:00, 376MB/s]


26-02-14 15:40:30 - 🔗 age_model_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5 to /root/.deepface/weights/age_model_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /root/.deepface/weights/age_model_weights.h5

  0%|          | 0.00/539M [00:00<?, ?B/s]
  2%|▏         | 11.0M/539M [00:00<00:05, 91.3MB/s]
  6%|▌         | 30.4M/539M [00:00<00:03, 146MB/s] 
  8%|▊         | 45.6M/539M [00:00<00:04, 122MB/s]
 11%|█         | 59.2M/539M [00:00<00:03, 127MB/s]
 14%|█▎        | 73.9M/539M [00:00<00:04, 116MB/s]
 16%|█▋        | 88.1M/539M [00:00<00:03, 123MB/s]
 19%|█▉        | 103M/539M [00:00<00:03, 129MB/s] 
 22%|██▏       | 116M/539M [00:00<00:03, 120MB/s]
 24%|██▍       | 130M/539M [00:01<00:03, 124MB/s]
 27%|██▋       | 147M/539M [00:01<00:02, 136MB/s]
 30%|██▉       | 161M/539M [00:01<00:03, 124MB/s]
 33%|███▎      | 176M/539M [00:01<00:02, 130MB/s]
 35%|███▌      | 189M/539M [00:01<00:02, 120MB/s]
 38%|███▊      | 203M/539M [00:01<00:02, 126MB/s]
 40%|████      | 218M/539M [00:01<00:02, 129MB/s]
 43%|████▎     | 231M/539M [00:01<00:02

26-02-14 15:40:40 - 🔗 gender_model_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5 to /root/.deepface/weights/gender_model_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: /root/.deepface/weights/gender_model_weights.h5

  0%|          | 0.00/537M [00:00<?, ?B/s]
  2%|▏         | 11.0M/537M [00:00<00:06, 76.8MB/s]
  5%|▍         | 26.2M/537M [00:00<00:04, 113MB/s] 
  8%|▊         | 42.5M/537M [00:00<00:04, 116MB/s]
 10%|█         | 55.6M/537M [00:00<00:03, 121MB/s]
 13%|█▎        | 69.7M/537M [00:00<00:03, 127MB/s]
 16%|█▌        | 84.4M/537M [00:00<00:03, 121MB/s]
 18%|█▊        | 99.1M/537M [00:00<00:03, 128MB/s]
 21%|██▏       | 115M/537M [00:00<00:03, 138MB/s] 
 24%|██▍       | 129M/537M [00:01<00:03, 128MB/s]
 27%|██▋       | 145M/537M [00:01<00:02, 136MB/s]
 30%|██▉       | 159M/537M [00:01<00:02, 127MB/s]
 32%|███▏      | 172M/537M [00:01<00:03, 98.4MB/s]
 35%|███▍      | 187M/537M [00:01<00:03, 109MB/s] 
 37%|███▋      | 200M/537M [00:01<00:03, 107MB/s]
 40%|████      | 217M/537M [00:01<00:02, 122MB/s]
 43%|████▎     | 231M/537M [00


Baigta!


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os

CSV_PATH = OUTPUT_CSV
OUT_DIR = "results"
os.makedirs(OUT_DIR, exist_ok=True)

df = pd.read_csv(CSV_PATH)

age_plot_path = os.path.join(OUT_DIR, f"age_hist_{run_id}.png")
gender_plot_path = os.path.join(OUT_DIR, f"gender_bar_{run_id}.png")

# 1) Amžiaus histograma
plt.figure()
df["age"].dropna().hist(bins=20)
plt.title("Amžiaus pasiskirstymas")
plt.xlabel("Amžius")
plt.ylabel("Kiekis")
plt.savefig(age_plot_path, bbox_inches="tight")
plt.close()

# 2) Lyčių pasiskirstymas
plt.figure()
df["gender"].value_counts().plot(kind="bar")
plt.title("Lyčių pasiskirstymas")
plt.xlabel("Lytis")
plt.ylabel("Kiekis")
plt.savefig(gender_plot_path, bbox_inches="tight")
plt.close()

In [5]:
from google.colab import files

print("Atsisiunčiami rezultatai...")
files.download(OUTPUT_VIDEO)
files.download(OUTPUT_CSV)
files.download(age_plot_path)
files.download(gender_plot_path)

Atsisiunčiami rezultatai...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>